In [ ]:
import os
import re
import sys
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from pyproj import CRS, Transformer
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline

if '..' not in sys.path:
    sys.path.append('..')
from pfcommon import build_network_graph, make_full_object_name, get_objects, Node, Edge

## Load grid data

In [ ]:
data_file = 'sardinia_geo.pkl'
force = False

if force or not os.path.isfile(data_file):
    powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2023 SP2\Python\3.10'
    if powerfactory_path not in sys.path:
        sys.path.append(powerfactory_path)
    import powerfactory as pf

    app = pf.GetApplication()
    if app is None:
        raise Exception('Cannot get PowerFactory application')
    print('Got PowerFactory application.')
    
    project_name = '\\Terna_Inerzia\\V2020_Rete_Sardegna_2021_06_03cr'
    err = app.ActivateProject(project_name)
    if err:
        raise Exception(f'Cannot activate project {project_name}')
    print(f'Activated project "{project_name}".')
    
    def get_coords(obj_type):
        objs = app.GetCalcRelevantObjects('*.' + obj_type)
        names, coords = [], []
        for obj in objs:
            if (obj.HasAttribute('outserv') and obj.outserv) or \
            not obj.HasAttribute('GPSlat') or not obj.HasAttribute('GPSlon'):
                continue
            lat,long = obj.GPSlat, obj.GPSlon
            if lat != 0.0 and long != 0.0:
                coords.append([lat,long])
                names.append(obj.loc_name)
        return names, np.array(coords)

    obj_types = {'sites': 'ElmSite', 'terminals': 'ElmTerm', 'substations': 'ElmSubstat'}
    names, coords = {}, {}
    for k,v in obj_types.items():
        names[k], coords[k] = get_coords(v)
    lines = [obj for obj in app.GetCalcRelevantObjects('*.ElmLne') if not obj.outserv]
    names['lines'], coords['lines'], line_ratings = [], [], []
    for line in lines:
        if len(line.GPScoords) > 0:
            names['lines'].append(line.loc_name)
            coords['lines'].append(np.array(line.GPScoords))
            line_ratings.append(line.typ_id.uline)
    coords['lines'] = np.array(coords['lines'], dtype=object)
    line_ratings = np.array(line_ratings)

    data = {
        'names': names,
        'coords': coords,
        'line_ratings': line_ratings
    }
    pickle.dump(data, open(data_file, 'wb'))
    
else:
    data = pickle.load(open(data_file, 'rb'))
    names = data['names']
    coords = data['coords']
    line_ratings = data['line_ratings']

## Coordinate reference systems

In [ ]:
# 3035: Lambert azimuthal equal area
# 3857: spherical Mercator projection
# 4326: world geodetic system 1984
source_coord_ref = 4326
coord_ref = 3857

Convert the coordinates of the grid to the chosen reference system:

In [ ]:
gdf = {key: gpd.GeoDataFrame(data={'name': names[key], 'geometry':
                             [Point(long,lat) for lat,long in
                              zip(coords[key][:,0], coords[key][:,1])]})
       for key in ('sites','substations','terminals')}
gdf['lines'] = gpd.GeoDataFrame(data={'name': names['lines'], 'vrating': line_ratings,
                                      'geometry': [LineString(np.array([line[:,1], line[:,0]]).T) 
                                                   for line in coords['lines']]})
vratings = np.unique(gdf['lines']['vrating'])[::-1]

for key in gdf:
    gdf[key].crs = CRS.from_user_input(source_coord_ref)
    gdf[key] = gdf[key].to_crs(epsg=coord_ref)
    gdf[key] = gdf[key].sort_values('name').set_index('name')

## Map of Sardinia

First define the bounding box:

In [ ]:
limits = {'WS': Point(8, 38.75), 'EN': Point(10, 41.4)}
bbox = gpd.GeoDataFrame(data=limits.values(),
                        index=pd.Index(data=limits.keys(), name='name'),
                        columns=['geometry'])
bbox.crs = CRS.from_user_input(source_coord_ref)
bbox = bbox.to_crs(epsg=coord_ref)

Then load the data and keep only those coordinates that fall within the bounding box:

In [ ]:
scale = 1 # 1 : 1,000,000
year = 2021
europe_folder = f'geography/ref-nuts-{year}-{scale:02d}m'
N_levels = 4
map_types = 'BN', #'LB' # BN: boundary, LB: label, RG: region
europe = {map_type: {} for map_type in map_types}
for level in range(N_levels):
    for map_type in map_types:
        if map_type == 'LB':
            europe_file = f'{europe_folder}/NUTS_{map_type}_{year}_{coord_ref}_LEVL_{level}.json'
        else:
            europe_file = f'{europe_folder}/NUTS_{map_type}_{scale:02d}M_{year}_{coord_ref}_LEVL_{level}.json'
        tmp = gpd.read_file(europe_file)
        tmp.crs = CRS.from_user_input(coord_ref)
        europe[map_type][level] = tmp.cx[bbox.loc['WS','geometry'].x : bbox.loc['EN','geometry'].x,
                                         bbox.loc['WS','geometry'].y : bbox.loc['EN','geometry'].y]

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(4,8))
light_gray = .8 + np.zeros(3)
dark_gray = .2 + np.zeros(3)
europe['BN'][0].plot(ax=ax, lw=1.25, color=dark_gray)
europe['BN'][3].plot(ax=ax, lw=0.5, color=light_gray)
for vrat in vratings:
    idx = gdf['lines']['vrating'] == vrat
    gdf['lines'].loc[idx,:].plot(ax=ax, lw=np.floor(vrat/100), color='tab:blue', label=f'{vrat:.0f} kV')
msizes = {'sites': 64, 'substations': 16, 'terminals': 4}
cmap = {'sites': [.5,1,.2], 'substations': [1,.25,1], 'terminals': [0,0,0]}
for type_name in cmap:
    gdf[type_name].plot(marker='o', ax=ax, markersize=msizes[type_name],
                        color=cmap[type_name], label=type_name)
ax.legend(loc='upper left', fontsize=8, frameon=False)
ax.axis('off')
fig.tight_layout()
plt.savefig('sardinia_geo_with_coords.pdf')

Plot Sardinia with overlaid the graph of the electrical network

In [ ]:
data = pickle.load(open('sardinia_graph.pkl', 'rb'))
edges = data['edges']
nodes = data['nodes']
G = nx.MultiGraph()
for e in edges:
    G.add_edge(e.node1.name, e.node2.name, weight=e.length, label=e.name, voltage=e.voltage)
voltages = np.unique([e.voltage for e in edges])[::-1]
lengths = np.unique([e.length for e in edges])[::-1]

In [ ]:
def fix_coords(source, graph, coords, depth_limit=100):
    found = False
    for i,edge in enumerate(nx.dfs_edges(graph, source=source.name, depth_limit=depth_limit)):
        for j,node in enumerate(edge):
            if coords[node][0] != 0 and coords[node][1] != 0:
                coords[source.name] = coords[node]
                return True
    return False

node_coords = {n.name: np.array([n.lon,n.lat]) for n in nodes}
for edge in edges:
    if np.any(edge.node1.coords == 0) and np.any(edge.node2.coords == 0):
        fix_coords(edge.node1, G, node_coords)
        node_coords[edge.node2.name] = node_coords[edge.node1.name]
    elif np.any(edge.node1.coords == 0):
        node_coords[edge.node1.name] = np.array([edge.node2.lon, edge.node2.lat])
    elif np.any(edge.node2.coords == 0):
        node_coords[edge.node2.name] = np.array([edge.node1.lon, edge.node2.lat])
        
source_crs = 'epsg:{}'.format(source_coord_ref)
target_crs = 'epsg:{}'.format(coord_ref)
trans = Transformer.from_crs(source_crs, target_crs)

for k in node_coords:
    lon,lat = node_coords[k]
    node_coords[k] = np.array(trans.transform(lat,lon))

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(4,8))
light_gray = .8 + np.zeros(3)
dark_gray = .2 + np.zeros(3)
europe['BN'][0].plot(ax=ax, lw=1.25, color=dark_gray)
europe['BN'][3].plot(ax=ax, lw=0.5, color=light_gray)

Vmin = 150
cmap = plt.get_cmap('Set1', (voltages>=Vmin).sum())
cmap = [[.8,0,0], [.8,0,.8], [0,.8,0]]
for i,V in enumerate(voltages):
    if V >= Vmin:
        idx = [(a,b) for a,b,d in G.edges(data=True) if d['voltage'] == V]
        lbl = '{:g} kV'.format(V)
    else:
        break
        idx = [(a,b) for a,b,d in G.edges(data=True) if d['voltage'] <= V]
        lbl = r'$\leq{:g}$ kV'.format(V)
    wdt = V / 70
    nx.draw_networkx_nodes(G.subgraph([jdx[0] for jdx in idx]), node_coords,
                           node_color='k', node_size=wdt*2, ax=ax)
    nx.draw_networkx_nodes(G.subgraph([jdx[1] for jdx in idx]), node_coords,
                           node_color='k', node_size=wdt*2, ax=ax)
    nx.draw_networkx_edges(G, node_coords, edgelist=idx, width=wdt, edge_color=cmap[i], label=lbl, ax=ax)
    if V < Vmin:
        break

ax.legend(loc='upper left', frameon=False, fontsize=8)
ax.axis('off')
fig.tight_layout()
plt.savefig('sardinia_geo_with_graph.pdf')